<a href="https://colab.research.google.com/github/abelarismendy/hackerrank/blob/master/examples/dreambooth/DreamBooth_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Sep 28 02:21:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [2]:
!git clone https://github.com/ShivamShrirao/diffusers
!pip install git+https://github.com/ShivamShrirao/diffusers             # 2 times for now cause colab wasn't able to find it for some reason. Will fix later prolly.
%pip install -U --pre triton
%pip install accelerate
%pip install transformers
%pip install ftfy
%pip install bitsandbytes

Cloning into 'diffusers'...
remote: Enumerating objects: 7135, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 7135 (delta 53), reused 75 (delta 52), pack-reused 7058
Receiving objects: 100% (7135/7135), 2.18 MiB | 23.50 MiB/s, done.
Resolving deltas: 100% (4666/4666), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ShivamShrirao/diffusers to /tmp/pip-req-build-swrwo6rk
  Running command git clone -q https://github.com/ShivamShrirao/diffusers /tmp/pip-req-build-swrwo6rk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 120 kB 33.6 MB/s 
  Created wheel for diffusers: filename=diffusers-0.4.0.dev0-py3-none-any.whl size=230053 sha256=0d562d0f64c31177ed2e096d1d928fc17d2e701a37d47db082ca09550dc3e624
  Stored in directory: /tm

### Install xformers from precompiled wheel. (These were compiled on Tesla T4, thanks to https://github.com/metrolobo)

In [3]:
%pip install https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

# if precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.0 MB 22 kB/s 


## Login to HF and run

In [4]:
%cd /content/diffusers/examples/dreambooth

/content/diffusers/examples/dreambooth


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [6]:
CLASS_NAME = "guy"          # just a general name for class like dog for dog images.

In [7]:
MODEL_NAME = "CompVis/stable-diffusion-v1-4"
INSTANCE_DIR = "/content/data/abel"            # upload your images in this directory, you can use the file manager on the left panel to upload.
!mkdir -p $INSTANCE_DIR
CLASS_DIR = f"/content/data/{CLASS_NAME}"
OUTPUT_DIR = "/content/models/abel"            # imv is a rare identifier, feel free to replace it.

In [10]:
# remove --use_8bit_adam flag if you got more than 18 GB VRAM.
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME --use_auth_token \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation \
  --instance_prompt="photo of imv {CLASS_NAME}" \
  --class_prompt="photo of a {CLASS_NAME}" \
  --resolution=512 \
  --use_8bit_adam \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=600

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Downloading: 100% 543/543 [00:00<00:00, 497kB/s]
Downloading: 100% 342/342 [00:00<00:00, 307kB/s]
Downloading: 100% 4.56k/4.56k [00:00<00:00, 3.71MB/s]
Downloading: 100% 1.22G/1.22G [00:18<00:00, 65.2MB/s]
Downloading: 100% 209/209 [00:00<00:00, 174kB/s]
Downloading: 100% 592/592 [00:00<00:00, 508kB/s]
Downloading: 100% 492M/492M [00:07<00:00, 69.8MB/s]
Downloading: 100% 525k/525k [00:00<00:00, 544kB/s]
Downloading: 100% 472/472 [00:00<00:00, 413kB/s]
Downloading: 100% 806/806 [00:00<00:00, 725kB/s]
Downloading: 100% 1.06M/1.06M [00:01<00:00, 737kB/s]
Downloading: 100% 743/743 

## Inference

In [11]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [12]:
pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.float16).to("cuda")

In [ ]:
batch_size = 4
prompt = ["portrait of imv guy thin face, black hair, round glasses, scar on forehead as a bolt of lightning, intricate, elegant, glowing lights, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by wlop, mars ravelo and greg rutkowski "] * batch_size

with autocast("cuda"):
    images = pipe(prompt, height=704, width=512, num_inference_steps=50, guidance_scale=13).images

for img in images:
    display(img)


  0%|          | 0/51 [00:00<?, ?it/s]